# Web scraping mobile.de with Selenium

The goal is to get information about cars behing sold in a specific area on the car reseller website mobile.de

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import time
import concurrent.futures

In [4]:
starttimer = time.perf_counter()

#how many sites do we need to go through
start_page = 1
end_page = 2

#creating list to save the data
list_ad_link = []
list_car_name = []
list_ad_date = []
list_price = []
list_phone_number = []
list_page = []

#go trhough the lit page of the car ads and collect some data
for nb_page in range(1,2):
    print(f'nb_page = {nb_page}')
    website = f"https://suchen.mobile.de/fahrzeuge/search.html?adLimitation=ONLY_FSBO_ADS&cn=DE&damageUnrepaired=NO_DAMAGE_UNREPAIRED&gn=65795,%20Hattersheim%20am%20Main,%20Hessen&grossPrice=true&isSearchRequest=true&ll=50.0535847,8.4848511&minPrice=7000&pageNumber={nb_page}&rd=50&ref=srpNextPage&refId=a592a669-58ef-4d57-bdcb-3d5bea50e3ac&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=ASCENDING"
    path = "C:/Users/F/PycharmProjects/chromedriver.exe"
    driver = webdriver.Chrome(path)
    driver.get(website)

    #get link to every ad from a page
    list_ad_link_period = []
    website1 = driver.find_elements(By.XPATH, '/html/body/div[4]/div[1]/div[3]/div[4]/div[2]/div[2]/div/a')
    for i in website1:
        list_ad_link.append(i.get_attribute('href'))
        list_ad_link_period.append(i.get_attribute('href'))
        #get car name from every ad from a page
    car_name = driver.find_elements(By.XPATH, '/html/body/div[4]/div[1]/div[3]/div[4]/div[2]/div[2]/div/a/div/div[2]/div[1]/div[1]/div/span[1]')
    for i in car_name:
        list_car_name.append(i.text)
    #get ad date
    ad_date = driver.find_elements(By.XPATH, '/html/body/div[4]/div[1]/div[3]/div[4]/div[2]/div[2]/div/a/div/div[2]/div[1]/div[1]/div/span[2]')
    for i in ad_date:
        iclean = i.text
        year = int(iclean[-11:-7])
        month = int(iclean[-14:-12])
        day = int(iclean[-17:-15])
        hour = int(iclean[-5:-3])
        minute = int(iclean[-2:])
        date = datetime(year, month, day, hour, minute)
        list_ad_date.append(date)

            
    #get price
    car_price = driver.find_elements(By.XPATH, '/html/body/div[4]/div[1]/div[3]/div[4]/div[2]/div[2]/div/a/div/div[2]/div[1]/div[2]/div[1]/span[1]')                           
    for i in car_price:         
        oh = i.text
        list_price.append(i.text)
    driver.quit()

    #go through the ad websites
    n = 0
    for i in list_ad_link_period:
        list_page.append(nb_page)
        ad_website = f"{i}"
        n = n + 1
        print(n)
        path = "C:/Users/F/PycharmProjects/chromedriver.exe"
        driver = webdriver.Chrome(path)
        driver.get(ad_website)
        try:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[20]/div/div[2]/div/button')))
        except:
            pass

        try:
            accept_button = driver.find_element(By.XPATH, '/html/body/div[20]/div/div[2]/div/button')               
            accept_button.click()
        except:
            pass
        #get phone number
        try:
            tel_number_button = driver.find_element(By.ID, 'sellerPhoneRevealButton')
            tel_number_button.click()                          
            tel_number = driver.find_element(By.ID, 'seller-phone')
            list_phone_number.append(tel_number.text)
        except:
            list_phone_number.append("no number")
            driver.quit()
            pass
 
driver.quit()
      

df = pd.DataFrame(columns=['page', 'ad name', 'ad date', 'price', 'phone number', 'ad link'])
df.loc[:,'page'] = list_page
df.loc[:,'ad name'] = list_car_name
df.loc[:,'ad date'] = list_ad_date
df.loc[:,'price'] = list_price
df.loc[:,'phone number'] = list_phone_number
df.loc[:,'ad link'] = list_ad_link
df.to_excel(f"mobile.de webscraping{start_page}.xlsx")
#return df



        
finishtimer = time.perf_counter()
print(f'finished in {round(finishtimer - starttimer,2)} seconds')

nb_page = 1


C:\Users\F\AppData\Local\Temp/ipykernel_9584/729907101.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


1


C:\Users\F\AppData\Local\Temp/ipykernel_9584/729907101.py:61: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


C:\Users\F\AppData\Local\Temp/ipykernel_9584/729907101.py:90: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'ad date'] = list_ad_date


finished in 519.5 seconds
